<a href="https://colab.research.google.com/github/kdj0712/study_data_analytics/blob/main/docs/quests/visuallizations/EDAs/multivariate_OrdersDeliveryList.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 한글폰트 설치 및 데이터 로드

In [ ]:
# 한글 폰트 설치
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 설치 후 colab 경우 Reuntime > Restart session 필요

In [8]:
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용

In [9]:
df_smdwd = pd.read_csv("/content/ShoppingMallDeliveryWithDate(2).csv")
df_smdwd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46749 entries, 0 to 46748
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   결제수단    46749 non-null  object 
 1   배송번호    46749 non-null  object 
 2   배송시작일   46749 non-null  object 
 3   배송완료일   46749 non-null  object 
 4   상품구매금액  46749 non-null  int64  
 5   상품번호    46661 non-null  float64
 6   수량      46749 non-null  int64  
 7   주문경로    46749 non-null  object 
 8   주문일     46749 non-null  int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 3.2+ MB


## 문제 풀이

###  1. 상품구매금액이 80,000원 이상인 주문들을 찾으세요.

In [10]:
df_smdwd_1 = df_smdwd.query('상품구매금액 >= 80000')
df_smdwd_1.info(), df_smdwd_1['상품구매금액'].min()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15595 entries, 1 to 46748
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   결제수단    15595 non-null  object 
 1   배송번호    15595 non-null  object 
 2   배송시작일   15595 non-null  object 
 3   배송완료일   15595 non-null  object 
 4   상품구매금액  15595 non-null  int64  
 5   상품번호    15579 non-null  float64
 6   수량      15595 non-null  int64  
 7   주문경로    15595 non-null  object 
 8   주문일     15595 non-null  int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 1.2+ MB


(None, 80000)

### 2. 모바일웹을 통해 주문된 상품 중, 상품구매금액이 평균 이상인 주문들을 찾으세요.

In [11]:
condition_path = '주문경로 == "모바일웹"'
df_smdwd_path = df_smdwd.query(condition_path)
condition_ave ='상품구매금액 >= 상품구매금액.mean()'
df_smdwd_2 = df_smdwd_path.query(condition_ave)
df_smdwd_path['상품구매금액'].mean(), df_smdwd_2.info(), df_smdwd_2['주문경로'].describe(include=object), df_smdwd_2['상품구매금액'].min()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8835 entries, 1 to 46748
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   결제수단    8835 non-null   object 
 1   배송번호    8835 non-null   object 
 2   배송시작일   8835 non-null   object 
 3   배송완료일   8835 non-null   object 
 4   상품구매금액  8835 non-null   int64  
 5   상품번호    8832 non-null   float64
 6   수량      8835 non-null   int64  
 7   주문경로    8835 non-null   object 
 8   주문일     8835 non-null   int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 690.2+ KB


(68979.56181533646,
 None,
 count     8835
 unique       1
 top       모바일웹
 freq      8835
 Name: 주문경로, dtype: object,
 69000)

### 3. 2019년 1월 7일 이전에 배송이 시작된 주문들을 찾으세요.

In [12]:
def convert_time(x):
    date, ampm, time = x.split()
    if ampm == "오후" and time != "12:00":
        hour, minute = map(int, time.split(":"))
        time = f"{hour+12 if hour != 12 else hour}:{minute}"
    return pd.to_datetime(f"{date} {time}", format="%Y-%m-%d %H:%M")

df_smdwd['배송시작일_date'] = df_smdwd['배송시작일'].apply(convert_time)

In [13]:
con_delivery = '배송시작일_date < "2019-01-07 00:00:00"'
df_smdwd_3 = df_smdwd.query(con_delivery)
df_smdwd_3.info() , df_smdwd_3['배송시작일_date'].max()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29498 entries, 1 to 46748
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   결제수단        29498 non-null  object        
 1   배송번호        29498 non-null  object        
 2   배송시작일       29498 non-null  object        
 3   배송완료일       29498 non-null  object        
 4   상품구매금액      29498 non-null  int64         
 5   상품번호        29410 non-null  float64       
 6   수량          29498 non-null  int64         
 7   주문경로        29498 non-null  object        
 8   주문일         29498 non-null  int64         
 9   배송시작일_date  29498 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(3), object(5)
memory usage: 2.5+ MB


(None, Timestamp('2019-01-03 18:20:00'))

### 4. 배송 시작일과 배송 완료일이 3일 이상 차이 나는 주문들을 찾으세요.

In [14]:
def convert_time(x):
    date, ampm, time = x.split()
    if ampm == "오후" and time != "12:00":
        hour, minute = map(int, time.split(":"))
        time = f"{hour+12 if hour != 12 else hour}:{minute}"
    return pd.to_datetime(f"{date} {time}", format="%Y-%m-%d %H:%M")

df_smdwd['배송완료일_date'] = df_smdwd['배송완료일'].apply(convert_time)

In [15]:
df_smdwd['배송소요일'] = df_smdwd['배송완료일_date'] - df_smdwd['배송시작일_date']
df_smdwd_3days = df_smdwd[df_smdwd['배송소요일'] >= pd.Timedelta(days=3)]
df_smdwd_3days.info(),df_smdwd_3days['배송소요일'].min()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46227 entries, 0 to 46748
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   결제수단        46227 non-null  object         
 1   배송번호        46227 non-null  object         
 2   배송시작일       46227 non-null  object         
 3   배송완료일       46227 non-null  object         
 4   상품구매금액      46227 non-null  int64          
 5   상품번호        46162 non-null  float64        
 6   수량          46227 non-null  int64          
 7   주문경로        46227 non-null  object         
 8   주문일         46227 non-null  int64          
 9   배송시작일_date  46227 non-null  datetime64[ns] 
 10  배송완료일_date  46227 non-null  datetime64[ns] 
 11  배송소요일       46227 non-null  timedelta64[ns]
dtypes: datetime64[ns](2), float64(1), int64(3), object(5), timedelta64[ns](1)
memory usage: 4.6+ MB


(None, Timedelta('3 days 00:22:00'))

### 5. 무통장입금을 결제수단으로 사용한 주문 중, 상품번호가 1000번 이상인 주문들을 찾으세요.

In [16]:
con_first = '결제수단 == "무통장입금"'
con_second = '상품번호 >= 1000'
conditions = f'{con_first} and {con_second}'
df_smdwd_5 = df_smdwd.query(conditions)
df_smdwd_5.info(), df_smdwd_5['결제수단'].describe(include=object), df_smdwd_5['상품번호'].min()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11769 entries, 0 to 45118
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   결제수단        11769 non-null  object         
 1   배송번호        11769 non-null  object         
 2   배송시작일       11769 non-null  object         
 3   배송완료일       11769 non-null  object         
 4   상품구매금액      11769 non-null  int64          
 5   상품번호        11769 non-null  float64        
 6   수량          11769 non-null  int64          
 7   주문경로        11769 non-null  object         
 8   주문일         11769 non-null  int64          
 9   배송시작일_date  11769 non-null  datetime64[ns] 
 10  배송완료일_date  11769 non-null  datetime64[ns] 
 11  배송소요일       11769 non-null  timedelta64[ns]
dtypes: datetime64[ns](2), float64(1), int64(3), object(5), timedelta64[ns](1)
memory usage: 1.2+ MB


(None,
 count     11769
 unique        1
 top       무통장입금
 freq      11769
 Name: 결제수단, dtype: object,
 1017.0)

### 6. 상품번호가 50번 이하이거나 1000번 이상인 주문들을 찾으세요.

In [81]:
df_smdwd_6 = df_smdwd.query('not (50 < 상품번호 < 1000)')
df_smdwd_6_1 = df_smdwd_6[df_smdwd_6['상품번호'].notnull()]
df_smdwd_6_1.info(), df_smdwd_6_1['상품번호'].describe(), df_smdwd_6_1['상품번호']

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24907 entries, 0 to 46733
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   결제수단        24907 non-null  object         
 1   배송번호        24907 non-null  object         
 2   배송시작일       24907 non-null  object         
 3   배송완료일       24907 non-null  object         
 4   상품구매금액      24907 non-null  int64          
 5   상품번호        24907 non-null  float64        
 6   수량          24907 non-null  int64          
 7   주문경로        24907 non-null  object         
 8   주문일         24907 non-null  int64          
 9   배송시작일_date  24907 non-null  datetime64[ns] 
 10  배송완료일_date  24907 non-null  datetime64[ns] 
 11  배송소요일       24907 non-null  timedelta64[ns]
 12  주문일_date    24907 non-null  datetime64[ns] 
dtypes: datetime64[ns](3), float64(1), int64(3), object(5), timedelta64[ns](1)
memory usage: 2.7+ MB


(None,
 count    24907.000000
 mean      1068.349460
 std        386.075495
 min         38.000000
 25%       1070.000000
 50%       1205.000000
 75%       1267.000000
 max       1681.000000
 Name: 상품번호, dtype: float64,
 0        1077.0
 1          42.0
 2        1271.0
 3        1085.0
 4        1106.0
           ...  
 46697      46.0
 46698      45.0
 46720      42.0
 46722      42.0
 46733      43.0
 Name: 상품번호, Length: 24907, dtype: float64)

In [49]:
if not df_smdwd_6['상품번호'].isin(range(51, 1000)).any():
    print("There are no values between 50 and 1000 in the '상품번호' column.")
else:
    print("There are values between 50 and 1000 in the '상품번호' column.")

There are no values between 50 and 1000 in the '상품번호' column.


### 7. 주문일이 20181231인 주문 중, 수량이 1개 이상인 주문들을 찾으세요.

In [33]:
df_smdwd['주문일_date'] = pd.to_datetime(df_smdwd['주문일'],format='%Y%m%d')
con_date = '주문일 == 20181231'
con_qty = '수량 >= 1'
con_order = f'{con_date} and {con_qty}'
df_smdwd_7 = df_smdwd.query(con_order)
df_smdwd_7.info() , df_smdwd_7['주문일_date'].describe(datetime_is_numeric=True), df_smdwd_7['수량'].describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 167 entries, 3 to 45117
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   결제수단        167 non-null    object         
 1   배송번호        167 non-null    object         
 2   배송시작일       167 non-null    object         
 3   배송완료일       167 non-null    object         
 4   상품구매금액      167 non-null    int64          
 5   상품번호        167 non-null    float64        
 6   수량          167 non-null    int64          
 7   주문경로        167 non-null    object         
 8   주문일         167 non-null    int64          
 9   배송시작일_date  167 non-null    datetime64[ns] 
 10  배송완료일_date  167 non-null    datetime64[ns] 
 11  배송소요일       167 non-null    timedelta64[ns]
 12  주문일_date    167 non-null    datetime64[ns] 
dtypes: datetime64[ns](3), float64(1), int64(3), object(5), timedelta64[ns](1)
memory usage: 18.3+ KB


(None,
 count                    167
 mean     2018-12-31 00:00:00
 min      2018-12-31 00:00:00
 25%      2018-12-31 00:00:00
 50%      2018-12-31 00:00:00
 75%      2018-12-31 00:00:00
 max      2018-12-31 00:00:00
 Name: 주문일_date, dtype: object,
 count    167.000000
 mean       1.125749
 std        0.427653
 min        1.000000
 25%        1.000000
 50%        1.000000
 75%        1.000000
 max        4.000000
 Name: 수량, dtype: float64)

### 8. PC쇼핑몰을 통해 주문된 상품 중, 배송번호가 'D-20181231'로 시작하는 주문들을 찾으세요

In [34]:
df_smdwd_newpath = df_smdwd.query('주문경로 == "PC쇼핑몰"')
df_smdwd_8 = df_smdwd_newpath[df_smdwd_newpath['배송번호'].str.contains('D-20181231')==True]
df_smdwd_8.info(),df_smdwd_8['주문경로'].describe(include=object), df_smdwd_8['배송번호'].max(),df_smdwd_8['배송번호'].min()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92 entries, 6 to 45105
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   결제수단        92 non-null     object         
 1   배송번호        92 non-null     object         
 2   배송시작일       92 non-null     object         
 3   배송완료일       92 non-null     object         
 4   상품구매금액      92 non-null     int64          
 5   상품번호        92 non-null     float64        
 6   수량          92 non-null     int64          
 7   주문경로        92 non-null     object         
 8   주문일         92 non-null     int64          
 9   배송시작일_date  92 non-null     datetime64[ns] 
 10  배송완료일_date  92 non-null     datetime64[ns] 
 11  배송소요일       92 non-null     timedelta64[ns]
 12  주문일_date    92 non-null     datetime64[ns] 
dtypes: datetime64[ns](3), float64(1), int64(3), object(5), timedelta64[ns](1)
memory usage: 10.1+ KB


(None,
 count        92
 unique        1
 top       PC쇼핑몰
 freq         92
 Name: 주문경로, dtype: object,
 'D-20181231-0000953-00',
 'D-20181231-0000025-00')

### 9. 상품구매금액이 70,000원 이상 100,000원 이하인 주문들 중, 수량이 1개인 주문들을 찾으세요.

In [35]:
con_fee = '100000 >= 상품구매금액 >= 70000'
con_quantity = '수량 == 1'
condition_9 = f"{con_fee} and {con_quantity}"
df_smdwd_9 = df_smdwd.query(condition_9)
df_smdwd_9.info(), df_smdwd_9['상품구매금액'].describe(), df_smdwd_9['수량'].describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12668 entries, 0 to 46748
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   결제수단        12668 non-null  object         
 1   배송번호        12668 non-null  object         
 2   배송시작일       12668 non-null  object         
 3   배송완료일       12668 non-null  object         
 4   상품구매금액      12668 non-null  int64          
 5   상품번호        12657 non-null  float64        
 6   수량          12668 non-null  int64          
 7   주문경로        12668 non-null  object         
 8   주문일         12668 non-null  int64          
 9   배송시작일_date  12668 non-null  datetime64[ns] 
 10  배송완료일_date  12668 non-null  datetime64[ns] 
 11  배송소요일       12668 non-null  timedelta64[ns]
 12  주문일_date    12668 non-null  datetime64[ns] 
dtypes: datetime64[ns](3), float64(1), int64(3), object(5), timedelta64[ns](1)
memory usage: 1.4+ MB


(None,
 count     12668.000000
 mean      81827.637354
 std        8132.625374
 min       70000.000000
 25%       75390.000000
 50%       78870.000000
 75%       87120.000000
 max      100000.000000
 Name: 상품구매금액, dtype: float64,
 count    12668.0
 mean         1.0
 std          0.0
 min          1.0
 25%          1.0
 50%          1.0
 75%          1.0
 max          1.0
 Name: 수량, dtype: float64)

### 10. 모바일웹을 통한 주문 중, 배송번호가 'D-20181230' 이후로 시작되고 상품구매금액이 50,000원 이상인 주문들을 찾으세요.

In [37]:
df_smdwd_mobile = df_smdwd.query('주문경로 == "모바일웹"')
df_smdwd_mobile_50000 = df_smdwd_mobile.query('상품구매금액 >= 50000')
df_smdwd_mobile_50000_10 = df_smdwd_mobile_50000.query('배송번호 > "D-20181230"')
df_smdwd_mobile_50000_10.info(), df_smdwd_mobile_50000_10['배송번호'].min(), df_smdwd_mobile_50000_10['상품구매금액'].min()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6474 entries, 3 to 45117
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   결제수단        6474 non-null   object         
 1   배송번호        6474 non-null   object         
 2   배송시작일       6474 non-null   object         
 3   배송완료일       6474 non-null   object         
 4   상품구매금액      6474 non-null   int64          
 5   상품번호        6474 non-null   float64        
 6   수량          6474 non-null   int64          
 7   주문경로        6474 non-null   object         
 8   주문일         6474 non-null   int64          
 9   배송시작일_date  6474 non-null   datetime64[ns] 
 10  배송완료일_date  6474 non-null   datetime64[ns] 
 11  배송소요일       6474 non-null   timedelta64[ns]
 12  주문일_date    6474 non-null   datetime64[ns] 
dtypes: datetime64[ns](3), float64(1), int64(3), object(5), timedelta64[ns](1)
memory usage: 708.1+ KB


(None, 'D-20181230-0000017-00', 50160)

### 11. 주문일이 2018년 12월 이후이고 상품구매금액이 50,000원 미만인 주문을 찾기.

In [38]:
df_smdwd['주문일_date']
df_smdwd_11 = df_smdwd.query('주문일_date > "2018-12-31" and 상품구매금액 < 50000')
df_smdwd_11.info(), df_smdwd_11['주문일_date'].min() , df_smdwd_11['상품구매금액'].max()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2891 entries, 41 to 17535
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   결제수단        2891 non-null   object         
 1   배송번호        2891 non-null   object         
 2   배송시작일       2891 non-null   object         
 3   배송완료일       2891 non-null   object         
 4   상품구매금액      2891 non-null   int64          
 5   상품번호        2891 non-null   float64        
 6   수량          2891 non-null   int64          
 7   주문경로        2891 non-null   object         
 8   주문일         2891 non-null   int64          
 9   배송시작일_date  2891 non-null   datetime64[ns] 
 10  배송완료일_date  2891 non-null   datetime64[ns] 
 11  배송소요일       2891 non-null   timedelta64[ns]
 12  주문일_date    2891 non-null   datetime64[ns] 
dtypes: datetime64[ns](3), float64(1), int64(3), object(5), timedelta64[ns](1)
memory usage: 316.2+ KB


(None, Timestamp('2019-01-01 00:00:00'), 49900)

### 12. 배송시작일이 2019년 1월 1일 이후이며, 결제수단이 '무통장입금'인 주문을 찾기.

In [39]:
cond_deli = '배송시작일_date > "2019-01-01 23:59:59"'
con_pay = '결제수단 == "무통장입금"'
con_12 = f"{cond_deli} and {con_pay}"
df_smdwd_12 = df_smdwd.query(con_12)
df_smdwd_12.info(), df_smdwd_12['배송시작일_date'].min(), df_smdwd_12['결제수단'].describe(include=object)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9076 entries, 0 to 45118
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   결제수단        9076 non-null   object         
 1   배송번호        9076 non-null   object         
 2   배송시작일       9076 non-null   object         
 3   배송완료일       9076 non-null   object         
 4   상품구매금액      9076 non-null   int64          
 5   상품번호        9076 non-null   float64        
 6   수량          9076 non-null   int64          
 7   주문경로        9076 non-null   object         
 8   주문일         9076 non-null   int64          
 9   배송시작일_date  9076 non-null   datetime64[ns] 
 10  배송완료일_date  9076 non-null   datetime64[ns] 
 11  배송소요일       9076 non-null   timedelta64[ns]
 12  주문일_date    9076 non-null   datetime64[ns] 
dtypes: datetime64[ns](3), float64(1), int64(3), object(5), timedelta64[ns](1)
memory usage: 992.7+ KB


(None,
 Timestamp('2019-01-03 09:28:00'),
 count      9076
 unique        1
 top       무통장입금
 freq       9076
 Name: 결제수단, dtype: object)

### 13. 상품번호가 1000 이상 2000 미만인 주문 중에서 수량이 2개 이상인 주문을 찾기.

In [40]:
item_num = '2000 > 상품번호 >= 1000'
qty_2 = "수량 >= 2"
con_13 = f"{item_num} and {qty_2}"
df_smdwd_13 = df_smdwd.query(con_13)
df_smdwd_13.info(), df_smdwd_13['상품번호'].describe(), df_smdwd_13['수량'].describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2781 entries, 8 to 45118
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   결제수단        2781 non-null   object         
 1   배송번호        2781 non-null   object         
 2   배송시작일       2781 non-null   object         
 3   배송완료일       2781 non-null   object         
 4   상품구매금액      2781 non-null   int64          
 5   상품번호        2781 non-null   float64        
 6   수량          2781 non-null   int64          
 7   주문경로        2781 non-null   object         
 8   주문일         2781 non-null   int64          
 9   배송시작일_date  2781 non-null   datetime64[ns] 
 10  배송완료일_date  2781 non-null   datetime64[ns] 
 11  배송소요일       2781 non-null   timedelta64[ns]
 12  주문일_date    2781 non-null   datetime64[ns] 
dtypes: datetime64[ns](3), float64(1), int64(3), object(5), timedelta64[ns](1)
memory usage: 304.2+ KB


(None,
 count    2781.000000
 mean     1228.533981
 std       135.736931
 min      1017.000000
 25%      1077.000000
 50%      1266.000000
 75%      1373.000000
 max      1625.000000
 Name: 상품번호, dtype: float64,
 count    2781.000000
 mean        2.432219
 std         0.920290
 min         2.000000
 25%         2.000000
 50%         2.000000
 75%         3.000000
 max        10.000000
 Name: 수량, dtype: float64)

### 14. 배송완료일이 배송시작일보다 5일 이상 차이 나는 주문을 찾기.

In [41]:
df_smdwd_14 = df_smdwd[df_smdwd['배송소요일'] >= pd.Timedelta(days=5)]
df_smdwd_14.info(),df_smdwd_14['배송소요일'].describe(include=datetime),df_smdwd_14['배송소요일'].min()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2509 entries, 41 to 46746
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   결제수단        2509 non-null   object         
 1   배송번호        2509 non-null   object         
 2   배송시작일       2509 non-null   object         
 3   배송완료일       2509 non-null   object         
 4   상품구매금액      2509 non-null   int64          
 5   상품번호        2448 non-null   float64        
 6   수량          2509 non-null   int64          
 7   주문경로        2509 non-null   object         
 8   주문일         2509 non-null   int64          
 9   배송시작일_date  2509 non-null   datetime64[ns] 
 10  배송완료일_date  2509 non-null   datetime64[ns] 
 11  배송소요일       2509 non-null   timedelta64[ns]
 12  주문일_date    2509 non-null   datetime64[ns] 
dtypes: datetime64[ns](3), float64(1), int64(3), object(5), timedelta64[ns](1)
memory usage: 274.4+ KB


(None,
 count                         2509
 mean     7 days 19:54:23.937823834
 std      2 days 22:52:57.283540063
 min                5 days 00:04:00
 25%                5 days 17:46:00
 50%                7 days 20:35:00
 75%                8 days 15:06:00
 max               46 days 18:06:00
 Name: 배송소요일, dtype: object,
 Timedelta('5 days 00:04:00'))

### 15. 주문경로가 '모바일웹'이면서 상품구매금액이 70,000원 이상 100,000원 미만인 주문을 찾기.

In [42]:
path_15 = '주문경로 == "모바일웹"'
fee_15 = '100000 > 상품구매금액 >= 70000'
con_15 = f"{path_15} and {fee_15}"
df_smdwd_15 = df_smdwd.query(con_15)
df_smdwd_15.info(),df_smdwd_15['주문경로'].describe(include=object), df_smdwd_15['상품구매금액'].describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5111 entries, 3 to 46748
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   결제수단        5111 non-null   object         
 1   배송번호        5111 non-null   object         
 2   배송시작일       5111 non-null   object         
 3   배송완료일       5111 non-null   object         
 4   상품구매금액      5111 non-null   int64          
 5   상품번호        5110 non-null   float64        
 6   수량          5111 non-null   int64          
 7   주문경로        5111 non-null   object         
 8   주문일         5111 non-null   int64          
 9   배송시작일_date  5111 non-null   datetime64[ns] 
 10  배송완료일_date  5111 non-null   datetime64[ns] 
 11  배송소요일       5111 non-null   timedelta64[ns]
 12  주문일_date    5111 non-null   datetime64[ns] 
dtypes: datetime64[ns](3), float64(1), int64(3), object(5), timedelta64[ns](1)
memory usage: 559.0+ KB


(None,
 count     5111
 unique       1
 top       모바일웹
 freq      5111
 Name: 주문경로, dtype: object,
 count     5111.000000
 mean     82271.036979
 std       8357.185098
 min      70000.000000
 25%      75570.000000
 50%      79900.000000
 75%      87410.000000
 max      99700.000000
 Name: 상품구매금액, dtype: float64)

### 16. 주문일이 20181230 이전이며, 배송번호가 'D-20181229' 이후로 시작하는 주문을 찾기.

In [43]:
order_16 = '주문일_date < "2018-12-30"'
delivery_16 = '배송번호 > "D-20181229"'
con_16 = f"{order_16} and {delivery_16}"
df_smdwd_16 = df_smdwd.query(con_16)
df_smdwd_16.info(), df_smdwd_16['주문일_date'].max(), df_smdwd_16["배송번호"].describe(include=object)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 1 to 44937
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   결제수단        50 non-null     object         
 1   배송번호        50 non-null     object         
 2   배송시작일       50 non-null     object         
 3   배송완료일       50 non-null     object         
 4   상품구매금액      50 non-null     int64          
 5   상품번호        50 non-null     float64        
 6   수량          50 non-null     int64          
 7   주문경로        50 non-null     object         
 8   주문일         50 non-null     int64          
 9   배송시작일_date  50 non-null     datetime64[ns] 
 10  배송완료일_date  50 non-null     datetime64[ns] 
 11  배송소요일       50 non-null     timedelta64[ns]
 12  주문일_date    50 non-null     datetime64[ns] 
dtypes: datetime64[ns](3), float64(1), int64(3), object(5), timedelta64[ns](1)
memory usage: 5.5+ KB


(None,
 Timestamp('2018-12-29 00:00:00'),
 count                        50
 unique                       29
 top       D-20181229-0000158-00
 freq                          2
 Name: 배송번호, dtype: object)

### 17. 상품번호가 50 이하이거나 1000 이상인 주문 중에서 주문경로가 'PC쇼핑몰'인 주문을 찾기.

In [44]:
path_17 = '주문경로 == "PC쇼핑몰"'
df_smdwd_17 = df_smdwd.query(path_17)
num_17_1 = '상품번호 <= 50'
num_17_2 = '상품번호 >= 1000'
con_17_1 = f"{num_17_1} or {num_17_2}"
df_smdwd_17_1 = df_smdwd_17.query(con_17_1)
df_smdwd_17_1.info(), df_smdwd_17_1['상품번호'].describe(), df_smdwd_17_1['주문경로'].describe(include=object)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13984 entries, 0 to 46722
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   결제수단        13984 non-null  object         
 1   배송번호        13984 non-null  object         
 2   배송시작일       13984 non-null  object         
 3   배송완료일       13984 non-null  object         
 4   상품구매금액      13984 non-null  int64          
 5   상품번호        13984 non-null  float64        
 6   수량          13984 non-null  int64          
 7   주문경로        13984 non-null  object         
 8   주문일         13984 non-null  int64          
 9   배송시작일_date  13984 non-null  datetime64[ns] 
 10  배송완료일_date  13984 non-null  datetime64[ns] 
 11  배송소요일       13984 non-null  timedelta64[ns]
 12  주문일_date    13984 non-null  datetime64[ns] 
dtypes: datetime64[ns](3), float64(1), int64(3), object(5), timedelta64[ns](1)
memory usage: 1.5+ MB


(None,
 count    13984.000000
 mean      1062.955521
 std        388.760298
 min         38.000000
 25%       1059.000000
 50%       1203.000000
 75%       1266.000000
 max       1681.000000
 Name: 상품번호, dtype: float64,
 count     13984
 unique        1
 top       PC쇼핑몰
 freq      13984
 Name: 주문경로, dtype: object)

In [45]:
if not df_smdwd_17_1['상품번호'].isin(range(51, 1000)).any():
    print("There are no values between 50 and 1000 in the '상품번호' column.")
else:
    print("There are values between 50 and 1000 in the '상품번호' column.")
df_smdwd_17_1['주문경로'].unique()
df_smdwd_17_1['상품번호'].values

There are no values between 50 and 1000 in the '상품번호' column.


array([1077., 1071., 1373., ...,   45.,   42.,   42.])

### 18. 배송번호가 'D-20181231'로 시작하는 주문 중, 배송완료일이 2019년 1월 7일 이후인 주문을 찾기.

In [46]:
delivery_18 = df_smdwd[df_smdwd['배송번호'].str.contains('D-20181231')==True]
order_18 = '배송완료일_date > "2019-01-07 23:59:59"'
df_smdwd_18 = delivery_18.query(order_18)
df_smdwd_18.info(), df_smdwd_18['배송완료일_date'].min(), df_smdwd_18["배송번호"].max()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4 entries, 4 to 45106
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   결제수단        4 non-null      object         
 1   배송번호        4 non-null      object         
 2   배송시작일       4 non-null      object         
 3   배송완료일       4 non-null      object         
 4   상품구매금액      4 non-null      int64          
 5   상품번호        4 non-null      float64        
 6   수량          4 non-null      int64          
 7   주문경로        4 non-null      object         
 8   주문일         4 non-null      int64          
 9   배송시작일_date  4 non-null      datetime64[ns] 
 10  배송완료일_date  4 non-null      datetime64[ns] 
 11  배송소요일       4 non-null      timedelta64[ns]
 12  주문일_date    4 non-null      datetime64[ns] 
dtypes: datetime64[ns](3), float64(1), int64(3), object(5), timedelta64[ns](1)
memory usage: 448.0+ bytes


(None, Timestamp('2019-01-09 14:47:00'), 'D-20181231-0000960-00')

### 19. 주문일이 20181231이고, 수량이 2개 이상인 주문을 찾기.

In [47]:
order_19 = '주문일_date == "2018-12-31"'
qtt_19 = '수량 >= 2'
con_19 = f"{order_19} and {qtt_19}"
df_smdwd_19 = df_smdwd.query(con_19)
df_smdwd_19.info(), df_smdwd_19['주문일_date'].describe(datetime_is_numeric=True), df_smdwd_19['수량'].describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17 entries, 8 to 45116
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   결제수단        17 non-null     object         
 1   배송번호        17 non-null     object         
 2   배송시작일       17 non-null     object         
 3   배송완료일       17 non-null     object         
 4   상품구매금액      17 non-null     int64          
 5   상품번호        17 non-null     float64        
 6   수량          17 non-null     int64          
 7   주문경로        17 non-null     object         
 8   주문일         17 non-null     int64          
 9   배송시작일_date  17 non-null     datetime64[ns] 
 10  배송완료일_date  17 non-null     datetime64[ns] 
 11  배송소요일       17 non-null     timedelta64[ns]
 12  주문일_date    17 non-null     datetime64[ns] 
dtypes: datetime64[ns](3), float64(1), int64(3), object(5), timedelta64[ns](1)
memory usage: 1.9+ KB


(None,
 count                     17
 mean     2018-12-31 00:00:00
 min      2018-12-31 00:00:00
 25%      2018-12-31 00:00:00
 50%      2018-12-31 00:00:00
 75%      2018-12-31 00:00:00
 max      2018-12-31 00:00:00
 Name: 주문일_date, dtype: object,
 count    17.000000
 mean      2.235294
 std       0.664211
 min       2.000000
 25%       2.000000
 50%       2.000000
 75%       2.000000
 max       4.000000
 Name: 수량, dtype: float64)

### 20. 배송시작일이 2019년 1월 3일인 주문 중, 상품구매금액이 평균 이하인 주문을 찾기.

In [48]:
con_20_1 = df_smdwd[(df_smdwd['배송시작일_date'].dt.year == 2019) & (df_smdwd['배송시작일_date'].dt.month == 1) & (df_smdwd['배송시작일_date'].dt.day == 3)]
con_20_2 = '상품구매금액 <= 상품구매금액.mean()'
df_smdwd_20 = con_20_1.query(con_20_2)

df_smdwd_20.info(),con_20_1['상품구매금액'].mean(), df_smdwd_20['상품구매금액'].max(), df_smdwd_20['배송시작일_date'].describe(datetime_is_numeric=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 234 entries, 2 to 45117
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype          
---  ------      --------------  -----          
 0   결제수단        234 non-null    object         
 1   배송번호        234 non-null    object         
 2   배송시작일       234 non-null    object         
 3   배송완료일       234 non-null    object         
 4   상품구매금액      234 non-null    int64          
 5   상품번호        234 non-null    float64        
 6   수량          234 non-null    int64          
 7   주문경로        234 non-null    object         
 8   주문일         234 non-null    int64          
 9   배송시작일_date  234 non-null    datetime64[ns] 
 10  배송완료일_date  234 non-null    datetime64[ns] 
 11  배송소요일       234 non-null    timedelta64[ns]
 12  주문일_date    234 non-null    datetime64[ns] 
dtypes: datetime64[ns](3), float64(1), int64(3), object(5), timedelta64[ns](1)
memory usage: 25.6+ KB


(None,
 99696.843575419,
 99660,
 count                              234
 mean     2019-01-03 13:29:11.025640704
 min                2019-01-03 09:28:00
 25%                2019-01-03 09:30:00
 50%                2019-01-03 09:30:00
 75%                2019-01-03 18:19:00
 max                2019-01-03 18:20:00
 Name: 배송시작일_date, dtype: object)